Wahlpflichtfach Künstliche Intelligenz I: Praktikum

---

# 07 - Übungsaufgaben Data Cleaning und Data Preparation

- [Aufgabe 1 - Cleaning und Preparation von medizinischen und geografischen Daten](#Aufgabe-1---Cleaning-und-Preparation-von-medizinischen-und-geografischen-Daten)
- [Aufgabe 2 - Data Preparation des Titanic-Datensatzes](#Aufgabe-2---Data-Preparation-des-Titanic-Datensatzes)

Dies ist das Übungsblatt zum Notebook [**07 - Data Cleaning und Data Preparation**](07%20-%20Data%20Cleaning%20und%20Data%20Preparation.ipynb)

## Aufgabe 1 - Cleaning und Preparation von medizinischen und geografischen Daten

In dieser Woche Übung versuchen wir, einen medizinischen Datensatz aufzuräumen und einen anderen geografischen Datensatz neu zu kombinieren. Der medizinische Datensatz ist ein Bericht über die Anzahl der Fälle und Todesfälle in verschiedenen Ländern aufgrund von Ebola. Sie finden das Dataset in `ebola_country_timeseries.csv`. Die Spalte `Date` ist ein Verweis auf das Datum der Beobachtung. Die Spalte `Day` ist ein Verweis auf den Tag der Beobachtung, beginnend bei 0 mit der ersten Beobachtung. Alle anderen Spalten enthalten die Anzahl der Fälle oder Todesfälle aufgrund von Ebola für ein bestimmtes Land.

a) **Machen Sie sich mit dem Datensatz vertraut (ohne automatische Evaluation):** Dies ist der erste Schritt, den Sie für jeden Datensatz, mit dem Sie arbeiten, tun sollten. Laden Sie den Datensatz mit Pandas. Schauen Sie sich seinen `head()` an, seine Spalten, und lassen Sie pandas den Datensatz für Sie mit `describe()` beschreiben. Wenn Sie etwas über einzelne Spalten wissen wollen, können Sie für diese zum Beispiel auch `value_counts()` verwenden.

In [ ]:
### BEGIN SOLUTION
import pandas as pd
df = pd.read_csv('../data/07/ebola_country_timeseries.csv')
df.describe(include="all")
### END SOLUTION

b) **Füllen Sie fehlende Werte auf:** Der Ebola-Datensatz enthält viele fehlende Werte für Daten, an denen die Anzahl der Fälle und Todesfälle nicht ermittelt werden konnte. Schreiben Sie eine Funktion `fill_nans(ebola_data)`, um das zu beheben. Da wir es mit einer Zeitreihe zu tun haben, nehmen wir an, dass der Wert einfach derselbe bleibt wie am letzten Datum, an dem eine Messung erhalten wurde. Sortieren Sie die Werte zunächst vom frühen zum späten Tag. Verwenden Sie dann die Vorwärtsfüllmethode, um die fehlenden Werte aufzufüllen. Es werden einige fehlende Werte an der Spitze übrig bleiben. Füllen Sie diese mit dem Median der jeweiligen Spalte auf. Schauen Sie sich noch einmal den Vortrag an, wie Sie dies mit Hilfe von Pandas-Primitiven einfach machen können.

In [ ]:
### BEGIN SOLUTION
def fill_nans(ebola_data):
    ebola_data_sorted = ebola_data.sort_values(by='Day', ascending=True)
    ebola_data_filled = ebola_data_sorted.fillna(method='ffill')
    ebola_data_completed = ebola_data_filled.fillna(ebola_data_filled.median(numeric_only=True))
    return ebola_data_completed
### END SOLUTION

In [ ]:
import pandas as pd
from pandas.testing import assert_frame_equal

df = pd.read_csv('../data/07/ebola_country_timeseries.csv')
result = fill_nans(df)

assert isinstance(result, pd.DataFrame), 'Your function needs to return a DataFrame.'
expected = pd.read_csv('../data/07/ebola_no_nans.csv', index_col=0)
assert_frame_equal(result.reset_index(drop=True).sort_index(axis=1),
                   expected.reset_index(drop=True).sort_index(axis=1))

c) **Wandeln Sie Spaltenüberschriften in Variablen um:** Schreiben Sie eine Funktion `tidy(ebola_data_no_nans)`, um den ebola-Datensatz zu bereinigen. Ihre Funktion erhält den Datenrahmen, der die Ausgabe Ihrer Funktion aus Aufgabe 1 sein soll. Dieser ist jedoch in der Datei `ebola_no_nans.csv` gespeichert, sodass Sie Aufgabe 1 nicht lösen "müssen", um Aufgabe 2 zu erledigen.

**Hinweis:** Der Ebola-Datensatz ist ein klassisches Beispiel für einen Datensatz, der Spaltenüberschriften als Ort zum Speichern von Variablen verwendet. Lassen Sie uns diesen Missbrauch in zwei Schritten bekämpfen. Verschmelzen Sie zunächst alle Spalten außer `Date` und `Day` in eine `value`-Spalte namens `count`- und eine `variable`-Spalte. Dann teilen Sie die neue `variable`-Spalte in eine `status`- und eine `country`-Spalte auf. Löschen Sie die alte `variable`-Spalte, die beim Verschmelzen entstanden ist. Schließlich sortieren Sie den Datenrahmen. Zuerst nach `Day`, dann nach `country` und nach `status`.

In [ ]:
### BEGIN SOLUTION
def tidy(ebola_data_no_nans):
    ebola_melted = (ebola_data_no_nans
       .melt(id_vars=['Date','Day'],
       var_name='variable', 
       value_name='count'))
    
    ebola_melted[['status','country']] = ebola_melted['variable'].str.split('_', expand=True)
    ebola_tidy = ebola_melted.drop(columns = 'variable')
    ebola_tidy = ebola_tidy.sort_values(by=['Day','country','status'], ascending=True)
    return ebola_tidy
### END SOLUTION

In [ ]:
df = pd.read_csv('../data/07/ebola_no_nans.csv', index_col=0)
expected = pd.read_csv('../data/07/ebola_tidy.csv', index_col=0)
result = tidy(df)
assert isinstance(result, pd.DataFrame), 'Your function needs to return a DataFrame.'
assert_frame_equal(result.reset_index(drop=True).sort_index(axis=1),
                   expected.reset_index(drop=True).sort_index(axis=1))

d) **Merging:** Diese Aufgabe konzentriert sich auf einen anderen Datensatz. Wir versuchen wieder, Bevölkerungsdichten zu finden, aber für einen anderen Datensatz in dem letzten Übungsblatt. Schreiben Sie eine Funktion `find_pop_density(pop, areas, abbreviations)`, die drei Dataframes als Parameter annimmt.

**Hinweis**: Der `pop`-DataFrame enthält die Bevölkerung verschiedener abgekürzter US-Bundesstaaten in verschiedenen Jahren. Zusätzlich enthält er die Information, ob die Zählung insgesamt oder nur für Personen unter 18 Jahren erfolgt. Die Daten sind in `state-population.csv` zu finden. Der DataFrame `areas` ordnet die vollständigen Namen der Bundesstaaten der jeweiligen Fläche in Quadratmeilen zu. Die Daten sind in `state-areas.csv` zu finden. Schließlich ordnet der DataFrame `abbreviations` die Abkürzungen der Bundesstaaten ihrem vollständigen Namen zu. Die Daten sind in `state-abbrevs.csv` zu finden. **Ihre Aufgabe besteht darin, die mittlere Gesamtbevölkerungsdichte pro Staat in Einwohner / Quadratkilometer zu finden. Der Umrechnungsfaktor von Quadratmeilen zu Quadratkilometern ist `1 / 0,38610` Sie können innere Joins verwenden, da wir uns in dieser Übung nicht um Werte kümmern, die nicht übereinstimmen. Ihr Endergebnis sollte eine `pd.Series` sein.**

In [ ]:
### BEGIN SOLUTION
def find_pop_density(pop, areas, abbreviations):
    areas_abbreviations = pd.merge(areas, abbreviations, on='state', how='inner')
    pop_density = areas_abbreviations
    pop_density = pd.merge(pop,areas_abbreviations, left_on='state/region', right_on='abbreviation', how='left')
    
    pop_density = pop_density[pop_density['ages'] == 'total']
    pop_density = pop_density.drop(columns = ['abbreviation','state/region','ages','year'])
    pop_density['area (sq. mi)'] = pop_density['area (sq. mi)'] / 0.38610
    pop_density['population'] = pop_density['population'] / pop_density['area (sq. mi)']
    pop_density = pop_density.groupby("state")["population"].mean()
    return pop_density
### END SOLUTION

In [ ]:
from pandas.testing import assert_series_equal
pop = pd.read_csv('../data/07/state-population.csv')
areas = pd.read_csv('../data/07/state-areas.csv')
abbreviations = pd.read_csv('../data/07/state-abbrevs.csv')

expected = pd.read_csv('../data/07/pop_density.csv', index_col=0, header=None).squeeze(1)
expected.index.name = 'state'
result = find_pop_density(pop, areas, abbreviations)
assert isinstance(result, pd.Series), 'Your function needs to return a Series.'
assert_series_equal(result, expected, check_names=False)

## Aufgabe 2 - Data Preparation des Titanic-Datensatzes

In Aufgabe a) und b) erkunden wir den Titanic-Datensatz, indem wir einige Spalten in kategoriale Spalten umwandeln und anschließend einige Pivot-Tabellen berechnen. Sie kennen den Titanic-Datensatz bereits aus dem Praktikums-Notebook. Er enthält Merkmale über Passagiere der tragischen Titanic-Fahrt. Eine detaillierte Beschreibung finden Sie hier https://www.kaggle.com/c/titanic/data.

In Aufgabe c) und d) kommen wir auf den Ebola-Datensatz zurück, den wir in Aufgabe 1 bereinigt haben. Wir werden unser neues Wissen über Zeitreihendaten nutzen, um den Datensatz richtig zu indizieren und zu glätten.

Alle Funktionen, die Sie schreiben und die einen DataFrame akzeptieren, sollten die Variable, die an sie übergeben wird, **nicht** verändern. Es kann also sinnvoll sein, den übergebenen DataFrame zunächst zu kopieren, bevor man Operationen darauf durchführt.

a) **Kategorische Spalten:** Der titanic-Datensatz enthält Spalten, die von Natur aus als kategorisch angesehen werden können, und andere, die durch Tiling in kategorische Spalten umgewandelt werden können. Schreiben Sie eine Funktion `categorize(titanic)`, die Folgendes tut:

1. Wandeln Sie die Spalte `Sex` in ungeordnete Kategorien um.
2. Wandeln Sie `Pclass` und `Embarked` in geordnete Kategorien um. Die Reihenfolge sollte 1 < 2 < 3 und S < C < Q sein (die Reihenfolge, in der die Titanic die Passagiere aufgenommen hat).
3. Verändern Sie `Parch` (die Anzahl der Eltern oder Kinder) in drei geordnete Kategorien um. Die erste Kategorie soll Passagiere enthalten, die keine Eltern oder Kinder an Bord hatten. Die zweite Kategorie diejenigen, die 1 oder 2 hatten, und die letzte alle, die mehr hatten. Nennen Sie die Kategorien jeweils "keine", "wenige" und "viele".
4. Bilden Sie aus der Spalte `Age` vier geordnete Kategorien, die auf den vier Quantilen der Altersverteilung basieren. Nennen Sie die Kategorien 'teenage', 'young', 'middleage' und 'senior'.

In [ ]:
### BEGIN SOLUTION
from pandas.api.types import CategoricalDtype

def categorize(df):
    pclass_dtype = CategoricalDtype(categories=[1, 2, 3], ordered=True)
    embarked_dtype = CategoricalDtype(categories=['S', 'C', 'Q'], ordered=True)
    parch_dtype = CategoricalDtype(categories=['keine', 'wenige', 'viele'], ordered=True)
    age_dtype = CategoricalDtype(categories=['teenager', 'young', 'middleage', "senior"], ordered=True)

    def _map_parch(parch):
        if parch == 0:
            return "keine"
        if parch <= 2:
            return "wenige"
        return "viele"

    df["Sex"] = df["Sex"].astype("category")
    df["Pclass"] = df["Pclass"].astype(pclass_dtype)
    df["Embarked"] = df["Embarked"].astype(embarked_dtype)
    df["Parch"] = df["Parch"].map(_map_parch).astype(parch_dtype)
    df["Age"] = pd.cut(df['Age'], bins=4, labels=['teenager', 'young', 'middleage', "senior"]).astype(age_dtype)


    return df
### END SOLUTION

In [ ]:
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal
import pickle

df = pd.read_csv('../data/07/titanic.csv')
df_before = df.copy()
result = categorize(df)
assert isinstance(result, pd.DataFrame), 'Your function needs to return a DataFrame.'
assert set(df["Sex"].cat.categories) == {"male", "female"}
assert set(df["Pclass"].cat.categories) == {1, 2, 3}
assert set(df["Embarked"].cat.categories) == {'S', 'C', 'Q'}
assert set(df["Parch"].cat.categories) == {'keine', 'wenige', 'viele'}
assert set(df["Age"].cat.categories) == {'teenager', 'young', 'middleage', 'senior'}

assert df["Pclass"].cat.ordered
assert df["Embarked"].cat.ordered
assert df["Parch"].cat.ordered
assert df["Age"].cat.ordered


b) **Pivot-Tabellen:** Schreiben Sie eine Funktion `pivot(titanic_cat)`, die zwei Pivot-Tabellen berechnet. Die erste Pivot-Tabelle soll `Embarked` gegen `Parch` sein und die mittlere Überlebensrate enthalten. Die zweite Pivot-Tabelle sollte `Sex` gegen `Age` sein, aber die **Anzahl** der Überlebenden in jeder dieser Gruppen enthalten. Überlegen Sie, welche Aggregationsfunktion Sie angesichts der Art und Weise, wie die Spalte `Survived` kodiert ist, verwenden können.

In [ ]:
df.head()

In [ ]:
### BEGIN SOLUTION
def pivot(df):
    result1 = df.pivot_table(values='Survived', index='Embarked', columns='Parch')
    result2 = df.pivot_table(values='Survived', index='Sex', columns='Age', aggfunc=["count"])

    return result1, result2
### END SOLUTION

In [ ]:
result1, result2 = pivot(df)
assert isinstance(result1, pd.DataFrame), 'Your function needs to return a DataFrame as first return value.'
assert isinstance(result2, pd.DataFrame), 'Your function needs to return a DataFrame as second return value.'

assert set(result1.columns.values) == {'keine', 'wenige', 'viele'}
assert set([n for _, n in result2.columns.values]) == {'teenager', 'young', 'middleage', 'senior'}
assert result1.values.max() <= 1 and result1.values.min() >= 0
assert result2.values.min() >= 1

c) **Zeitreihen-Indizierung:** Diese Aufgaben konzentrieren sich wieder auf den ebola-Datensatz. Schreiben Sie eine Funktion `index_by_time()`, die `ebola_tidy.csv` aus dem Dateisystem liest. Dies ist die aufgeräumte Version des ebola-Datensatzes, den wir beim letzten Mal erstellt haben. Verwenden Sie das `Date` als `DatetimeIndex`, d. h. als einen Index, der auf Timestamps basiert. Dann entfernen Sie die Spalte `Day` und behalten nur Beobachtungen, die zwischen August 2014 und Januar 2015 (einschließlich) gemacht wurden. Sortieren Sie schließlich nach dem Index.

In [ ]:
### BEGIN SOLUTION
def index_by_time():
    df = pd.read_csv('../data/07/ebola_tidy.csv', index_col=0)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date")
    df = df.drop(columns=["Day"])
    df = df.loc[(df.index >= "2014-08-01") & (df.index <= "2015-01-01")]    

    return df
### END SOLUTION

In [ ]:
from datetime import datetime

result_c = index_by_time()
assert isinstance(result_c, pd.DataFrame), 'Your function needs to return a DataFrame.'

assert result_c.index[0] >= datetime(2014, 8, 1)
assert result_c.index[-1] <= datetime(2015, 1, 1)

d) **Smoothing:** Nun wollen wir uns eine geglättete Version der Entwicklung der Anzahl der Ebola-Fälle in den verschiedenen Ländern ansehen. Filtern Sie zunächst die Beobachtungen über die Anzahl der Todesfälle heraus und behalten Sie nur die über die Fälle. Dann lassen Sie die Statusspalte weg. Als nächstes müssen Sie die Ländernamen wieder in Spalten pivotieren. Verwenden Sie dazu die `pivot`-Methode. Sie funktioniert auf die gleiche Weise wie `pivot_table`, berechnet aber keine Aggregationen. Stattdessen strukturiert sie die Daten einfach um. Im nächsten Schritt berechnen Sie schließlich einen gleitenden Mittelwert über alle vier Wochen, sodass die Glättung ungefähr über einen Monat erfolgt.

**Hinweis:** Auf das Endergebnis können Sie nun einfach `.plot()` aufrufen, um eine schöne Visualisierung zu erhalten!

In [ ]:
### BEGIN SOLUTION
def smoothing(df):
    df = df.loc[df["status"] == "Cases"]
    df = df.drop(columns=["status"])
    df = df.pivot(columns=["country"])
    return df.asfreq('4W', method='ffill')
    
### END SOLUTION

In [ ]:
result_d = smoothing(result_c)
assert isinstance(result_d, pd.DataFrame), 'Your function needs to return a DataFrame.'
assert result_d.index[0] >= datetime(2014, 8, 1)
assert result_d.index[-1] <= datetime(2015, 1, 1)
result_d.plot()

---

Wahlpflichtfach Künstliche Intelligenz I: Praktikum